In [1]:
from langchain_community.vectorstores import Chroma
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
 

In [2]:
docs = TextLoader("speech.txt").load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
final_docs = text_splitter.split_documents(docs)


In [3]:
embeddings = OllamaEmbeddings(model="gemma:2b")

db = Chroma.from_documents(documents=final_docs,embedding=embeddings)
db

/tmp/ipykernel_35739/2597325816.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="gemma:2b")


In [4]:
###query

query = "What does the speaker believe is the main reason the United States should enter the war?"
docs = db.similarity_search(query)
docs[0].page_content

'always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

In [5]:
##save to local

db = Chroma.from_documents(documents=final_docs,embedding=embeddings, persist_directory="./chroma_db")

In [6]:
#load from local/disk

db2 = Chroma(persist_directory="./chroma_db",embedding_function=embeddings)

In [7]:
docs = db2.similarity_search(query)
docs[0].page_content

'always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

In [8]:
docs = db2.similarity_search_with_score(query)
docs

[(Document(id='6ce9072e-12cc-484b-91de-44ff247a9fed', metadata={'source': 'speech.txt'}, page_content='always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
  3531.251953125),
 (Document(id='fa56317d-3c09-4794-b535-f23578c32969', metadata={'source': 'speech.txt'}, page_content='always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
  3531.251953125),
 (Document(id='99ddfb9b-6b83-4db5-a9de-870bd4f6265a', metadata={'sourc

In [9]:
###retriever option

retriever = db.as_retriever()
retriever.invoke(query)

[Document(id='6ce9072e-12cc-484b-91de-44ff247a9fed', metadata={'source': 'speech.txt'}, page_content='always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='fa56317d-3c09-4794-b535-f23578c32969', metadata={'source': 'speech.txt'}, page_content='always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='99ddfb9b-6b83-4db5-a9de-870bd4f6265a', metadata={'source': 'speech.txt'}, page_content='The worl

In [10]:
retriever.invoke(query)[0].page_content

'always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'